## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    today = dt.datetime.now()
    toDate = dt.datetime(2019,4,16,15,15)
    fromDate = toDate - dt.timedelta(days = 1, hours=4)
    mode = "Algo"
    papertrade = True


### Filter Stocks - portfolio maker

In [2]:
if mode == "Algo":
    portfolio = ['HDFCBANK']

##  <span style="color:green">Trading Strategy </span>

In [3]:
# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0

# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a, days: ROC(a,days) >= 0.01
#TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01

TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)))
TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2)))
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
     
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    
    def long_pa(self):
        return pd.DataFrame(TREND_DOWN(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
                            #| (CDLABANDONEDBABY(OPEN, HIGH, LOW, CLOSE) != 0) 
                            ), columns=["buy"])
    def short_pa(self):
        return pd.DataFrame( TREND_UP(HIGH, LOW) & 
                            ((CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
                            | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
                            #| (CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0)
                            ) , columns=["sell"])
    
    def tradeDecision(self):
        
        buy  = self.long_pa()   
        sell = self.short_pa()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    
    def tradeBot(self, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        global STATE, profit, squareoff, stoploss
        
        #logger.debug(symbol+","+STATE)
        timeNow = timestamp
        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)

        #logger.debug(timeNow)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = price - 2 * SD[-1] 
        squareoff = 3 * SD[-1] 
        
        
        #if (LOW[-1]<=stoploss) and (STATE =="SELL"):
        #    profit = profit - stoploss
        #    STATE = "BUY"
        #    return
        #
        #if (HIGH[-1]<=stoploss) and (STATE =="BUY"):
        #    profit = profit + stoploss
        #    STATE = "SELL"
        #    return
        ##
        #if HIGH[-1] >= squareoff:
        #    profit = profit - squareoff
        #    STATE = "BUY"
        #    return
        
        if (timeNow.time() > dt.time(11,30)) and (timeNow.time() < dt.time(14,0)):
            # Mostly Choppy period# Mostly Trending period
            buy  = self.long_bb()   
            sell = self.short_bb()

            buy = buy[buy['buy']]
            sell = sell[sell['sell']]
            
        else:
            # Mostly Trending period
            buy  = self.long_pa()   
            sell = self.short_pa()

            buy = buy[buy['buy']]
            sell = sell[sell['sell']]

        if buy.empty | sell.empty:
            return
        
        if ((delT1 == buy.index[-1]) or (delT2 == buy.index[-1])) and (STATE != "SELL"):
            if STATE == "":
                profit = profit - price
            else:
                profit = profit - 2*price
            
            logger.debug("buy:"+str(buy.index[-1]))
            
            ##### Order Placement #####
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
        elif ((delT1 == sell.index[-1]) | (delT2 == sell.index[-1])) and (STATE != "BUY"):
            if STATE == "":
                profit = profit + price
            else:
                profit = profit + 2*price
                
            logger.debug("sell:"+str(sell.index[-1]))
            
            ##### Order Placement #####
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, stoploss, quantity=1)
            STATE = "BUY"




In [ ]:
logger.setLevel(logging.DEBUG)
offset = 90

nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

nifty50list = nifty50['Symbol']

portfolio = pd.DataFrame(nifty50list)
papertrade = True
def papertrade(portfolio):
    for index,symbol in portfolio.iterrows():
        profit = 0
        STATE = "BUY"
        squareoff = 0
        stoploss = 0
        global fig
        noofcandles = 60
        x = symbol[0]
        temp_data = getData(x, fromDate = toDate - dt.timedelta(days=2, hours=9), toDate=toDate, freq="minute")
        rowSize = temp_data.shape[0]

        for i in np.linspace(offset,rowSize-1, rowSize-offset -1, dtype="int64"):
            if(toDate.date() != temp_data.index[i].date()):
                continue
            a = algoTrade(temp_data.iloc[(i-offset):i])
            
            a.tradeBot(x, temp_data.index[i].to_pydatetime())
            #print(temp_data.iloc[(i-30):i])
        if STATE == "SELL":
            profit = profit + 2*temp_data.iloc[-1].close

        print(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")            

In [64]:
   
papertrade = False
if (mode=="Algo"):
    global fig
    noofcandles = 60
    x = "ZEEL"
    temp_data = getData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    plot_set1(temp_data)
     
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()

    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)

Using cache: Not downloading data
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



## <span style="color:blue">BackTesting - Single/Portfolio</span>

In [20]:
#portfolio = pd.DataFrame(downloadlist)
logger.setLevel(logging.DEBUG)
offset = 90
portfolio = pd.DataFrame(['WIPRO'])
papertrade = True
if papertrade:
    for index,symbol in portfolio.iterrows():
        profit = 0
        STATE = "BUY"
        squareoff = 0
        stoploss = 0
        global fig
        noofcandles = 60
        x = symbol[0]
        temp_data = getData(x, fromDate = toDate - dt.timedelta(days=2, hours=9), toDate=toDate, freq="minute")
        rowSize = temp_data.shape[0]

        for i in np.linspace(offset,rowSize-1, rowSize-offset -1, dtype="int64"):
            if(toDate.date() != temp_data.index[i].date()):
                continue
            a = algoTrade(temp_data.iloc[(i-offset):i])
            
            a.tradeBot(x, temp_data.index[i].to_pydatetime())
            #print(temp_data.iloc[(i-30):i])
        if STATE == "SELL":
            profit = profit + 2*temp_data.iloc[-1].close

        print(x+": "+str(round(profit,2))+", ("+str(round(profit/temp_data.iloc[-1].close*100,4))+" %)")

Using cache: Not downloading data
2019-04-18 09:15:00 <class 'datetime.datetime'>
2019-04-18 09:16:00 <class 'datetime.datetime'>
2019-04-18 09:17:00 <class 'datetime.datetime'>
2019-04-18 09:18:00 <class 'datetime.datetime'>
2019-04-18 09:19:00 <class 'datetime.datetime'>
2019-04-18 09:20:00 <class 'datetime.datetime'>
2019-04-18 09:21:00 <class 'datetime.datetime'>
2019-04-18 09:22:00 <class 'datetime.datetime'>
2019-04-18 09:23:00 <class 'datetime.datetime'>
2019-04-18 09:24:00 <class 'datetime.datetime'>
2019-04-18 09:25:00 <class 'datetime.datetime'>
2019-04-18 09:26:00 <class 'datetime.datetime'>
2019-04-18 09:27:00 <class 'datetime.datetime'>
2019-04-18 09:28:00 <class 'datetime.datetime'>
2019-04-18 09:29:00 <class 'datetime.datetime'>
2019-04-18 09:30:00 <class 'datetime.datetime'>
2019-04-18 09:31:00 <class 'datetime.datetime'>
2019-04-18 09:32:00 <class 'datetime.datetime'>
2019-04-18 09:33:00 <class 'datetime.datetime'>
2019-04-18 09:34:00 <class 'datetime.datetime'>
2019-0

2019-04-18 12:13:00 <class 'datetime.datetime'>
2019-04-18 12:14:00 <class 'datetime.datetime'>
2019-04-18 12:15:00 <class 'datetime.datetime'>
2019-04-18 12:16:00 <class 'datetime.datetime'>
2019-04-18 12:17:00 <class 'datetime.datetime'>
2019-04-18 12:18:00 <class 'datetime.datetime'>
2019-04-18 12:19:00 <class 'datetime.datetime'>
2019-04-18 12:20:00 <class 'datetime.datetime'>
2019-04-18 12:21:00 <class 'datetime.datetime'>
2019-04-18 12:22:00 <class 'datetime.datetime'>
2019-04-18 12:23:00 <class 'datetime.datetime'>
2019-04-18 12:24:00 <class 'datetime.datetime'>
2019-04-18 12:25:00 <class 'datetime.datetime'>
2019-04-18 12:26:00 <class 'datetime.datetime'>
2019-04-18 12:27:00 <class 'datetime.datetime'>
2019-04-18 12:28:00 <class 'datetime.datetime'>
2019-04-18 12:29:00 <class 'datetime.datetime'>
2019-04-18 12:33:00 <class 'datetime.datetime'>
2019-04-18 12:34:00 <class 'datetime.datetime'>
2019-04-18 12:35:00 <class 'datetime.datetime'>
2019-04-18 12:36:00 <class 'datetime.dat

2019-04-18 15:07:00 <class 'datetime.datetime'>
2019-04-18 15:08:00 <class 'datetime.datetime'>
2019-04-18 15:09:00 <class 'datetime.datetime'>
2019-04-18 15:10:00 <class 'datetime.datetime'>
2019-04-18 15:11:00 <class 'datetime.datetime'>
2019-04-18 15:12:00 <class 'datetime.datetime'>
2019-04-18 15:13:00 <class 'datetime.datetime'>
2019-04-18 15:15:00 <class 'datetime.datetime'>
WIPRO: -3.7, (-1.2969 %)


- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick

## Batch Analysis - Multithreaded

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
threaded = True
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    fig = ""
    temp_data = downloadData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #print(temp_data.head())
    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    obv(temp_data,2, True)

    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData(x, True)

if __name__ == '__main__':
    #if True | (mode == "Algo"):
    logger.debug("Hello")
    p = Pool(10)
    p.map_async(threadedBatchAnalysis, portfolio[0])
    #p.apply_async(threadedBatchAnalysis, ['SBIN'])
    #portfolio
    
#threadedBatchAnalysis()

In [12]:
import gc
gc.get_count()

(415, 0, 0)

In [11]:
gc.collect()

19269

### Backtesting - Single Threaded

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
def batchAnalysis(portfolio=portfolio):
    global fig, noofcandles
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row[0] #row.iloc[:,0]
        temp_data = downloadData(symbol, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
        
        #temp_data['symbol'] = symbol
        #temp_data.set_index(['temp_data.index], inplace=True)
        #logging.debug(temp_data)
        
        yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = temp_data.index[-1*noofcandles]
        xMax = temp_data.index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

batchAnalysis()

NetworkException: Request preparation failed (kitetrade-common).